# Regression
Some straightforward reading on fixed effects can be done here: http://www.jblumenstock.com/files/courses/econ174/FEModels.pdf.

The following is from the paper:

... assume that the number of crimes $C_{iym}$ in month $m$ of year $y$ in county $i$ of state $s$ has a Poisson distribution with probability density function given by

$$
f\left(C_{iym}\middle|\mathbf{X}_{iym}\right) = \frac{\mu\left(\mathbf{X}_{iym}\right)^{C_{iym}}\exp{\left(-\mu\left(\mathbf{X}_{iym}\right)\right)}}{C_{iym}!}
$$

where $\mathbf{X}_{iym}$ is the set of all observed covariates and $\mu\left(\mathbf{X}_{iym}\right) \equiv \mathbb{E}\left[C_{iym}\middle|\mathbf{X}_{iym}\right]$ is a link function that provides a parametric form for the conditional mean of $C_{iym}$ given $\mathbf{X}_{iym}$ ... assume that $\mu\left(\mathbf{X}_{iym}\right)$ takes an exponetial form

$$
\mu\left(\mathbf{X}_{iym}\right) = \exp{\left(\sum_{j=1}^{11} \alpha_0^jT_{iym}^j + \sum_{k=1}^5\beta_0^kP_{iym}^k + \sum_{j=1}^{11} \alpha_1^jT_{i,y,m-1}^j + \sum_{k=1}^5\beta_1^kP_{i,y,m-1}^k + \mathit{\Phi}_{sm} + \theta_{iy}\right)}
$$

In this equation, $\mathit{\Phi}_{sm}$ is a state-by-month fixed effect and $\theta_{iy}$ is a county-by-year fixed effect ... model the daily distribution of temperatures within a month using 11 bin variables (in degrees fahrenheit): <10, 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99, >=100. For example, the variable $T_{iym}^j$ represents the number of days in month $m$ of year $y$ in county $i$ in which the temperature fell into temperature bin $j$ ... use a similar convention for the precipitation variables $P_{iym}^k$, with five bins: 0 mm, 1-4 mm, 5-14 mm, 15-29 mm, >=30 mm ... also include one month lag of each temperature and precipitation bin variable.

In [1]:
# consider note under coefficients (specifically removal of fixed effects "by conditioning on a sufficient statistic")

## Prepping Data for Regression

In [1]:
import numpy as np
import pandas as pd
import pickle
import ipyparallel
import datetime
import math
import itertools
from IPython.display import clear_output
from statsmodels.discrete.discrete_model import Poisson

C:\Users\avi\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [13]:
with open('pickles/fips_temp_df.pkl', 'rb') as file:
    fips_temp_df = pickle.load(file)

In [14]:
with open('pickles/fips_prcp_df.pkl', 'rb') as file:
    fips_prcp_df = pickle.load(file)

In [15]:
fips_temp_df.head()

,27077,53073,53047,53019,16021,30101,30105,30091,38023,38013,...,12099,12071,48427,12051,48489,48215,12021,12011,12086,12087
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,18.712920,36.685883,32.012584,25.857495,35.831735,45.079819,36.754023,30.515072,25.709490,24.98,...,67.618750,64.837862,67.282964,66.043967,64.04,66.407741,72.301503,59.431992,23.376757,56.614433
1980-01-02,11.071802,38.492549,31.016061,24.764633,37.158181,43.951076,24.790933,19.102348,16.391507,15.98,...,64.426049,63.727822,71.060000,64.250718,71.06,71.060000,68.241944,55.256260,21.693056,51.098822
1980-01-03,7.251181,35.231312,27.923699,24.240318,37.655729,37.492174,35.184564,28.116247,21.955243,21.02,...,63.711470,71.047895,77.000000,67.186110,77.00,77.000000,67.490662,60.519064,24.399004,55.907304
1980-01-04,16.446460,32.616167,25.005640,22.730669,32.033837,34.412249,24.238771,25.893898,22.443736,21.92,...,73.427716,69.110040,61.175555,69.793249,62.06,61.414252,72.059558,65.067724,25.721912,59.018674
1980-01-05,22.254658,30.571894,26.863950,23.584503,31.599524,28.359593,15.805631,16.641174,15.285754,15.08,...,68.551042,65.280600,65.135555,66.904356,66.02,65.374252,68.732165,56.866954,22.053849,53.503063


In [16]:
fips_prcp_df.head()

,27077,53073,53047,53019,16021,30101,30105,30091,38023,38013,...,12099,12071,48427,12051,48489,48215,12021,12011,12086,12087
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,-0.011909,7.332760,1.076370,2.315682,1.108069,-8.278127e-18,0.816060,9.940308e-18,1.651693e-17,0.0,...,-7.784454e-17,0.04738,7.122941e-18,6.885559e-02,0.0,1.073579e-17,9.788715e-19,0.003429,-0.034548,0.299835
1980-01-02,-0.011909,2.096022,1.820167,1.114001,2.378330,-8.278127e-18,0.011853,9.940308e-18,1.651693e-17,0.0,...,-7.784454e-17,0.00000,7.122941e-18,-3.650408e-17,0.0,1.073579e-17,9.788715e-19,0.003429,-0.034548,0.299835
1980-01-03,-0.011909,2.476945,0.812912,1.155021,1.911363,1.574522e-01,0.964153,9.230563e-01,2.292168e+00,2.5,...,-7.784454e-17,0.00000,4.164604e-01,-3.650408e-17,2.3,9.247968e-01,9.788715e-19,0.003429,-0.034548,0.299835
1980-01-04,-0.011909,0.361987,0.017442,0.033442,3.033882,-8.278127e-18,0.011853,5.634584e-01,7.688252e-01,0.8,...,7.351824e-01,1.58282,7.122941e-18,9.183971e-01,0.0,1.073579e-17,4.951725e-01,1.478177,0.633587,8.313971
1980-01-05,-0.011909,0.255781,1.411700,2.209812,8.767510,5.301087e+00,3.127516,9.940308e-18,1.651693e-17,0.0,...,-7.784454e-17,0.00000,7.122941e-18,-3.650408e-17,0.0,1.073579e-17,9.788715e-19,0.003429,-0.034548,0.299835


Appropriately transform temperature dataframe to easily allow dummy variable generation by multi-indexing by state fips code.

In [17]:
# transpose
temp_df = fips_temp_df.T
# sort indeces (we know that the first two numbers in the fips code id the state)
temp_df.sort_index(inplace=True)
# create a state column
temp_df = temp_df.assign(state=temp_df.index.str[:2])
# rename index to fips
temp_df.index.rename('fips', inplace=True)
# reset index in order to then create a multi-index consisting of state fips and state county fips
temp_df = temp_df.reset_index()
temp_df = temp_df.set_index(['state', 'fips'])
# re-transpose
temp_df = temp_df.T
# make index datetime
temp_df.index = pd.to_datetime(temp_df.index)

In [18]:
temp_df.head()

state              01                                                         \
fips            01001      01003      01005      01007      01009      01011   
date                                                                           
1980-01-01  51.213894  50.027057  47.386815  53.060000  46.816072  47.133440   
1980-01-02  56.713572  57.053789  50.760231  59.310918  48.049430  50.126903   
1980-01-03  61.462730  31.846381  57.227373  61.741007  44.326941  56.728705   
1980-01-04  59.409078  64.768273  60.652524  54.931063  53.913923  60.944206   
1980-01-05  44.960000  46.692099  46.787850  44.300072  43.222982  46.930683   

state                                                     ...             56  \
fips            01013      01015      01017      01019    ...          56027   
date                                                      ...                  
1980-01-01  49.871340  49.301943  45.782574  43.570334    ...      14.254621   
1980-01-02  56.897629  51.608675  49.207633  46.274620    ...      14.352464   
1980-01-03  62.818029  54.311243  59.122542  44.445283    ...      12.000208   
1980-01-04  60.241459  54.230422  56.851033  54.542617    ...      12.345711   
1980-01-05  45.299517  46.433862  47.368302  43.283155    ...      13.688148   

state                                                                         \
fips            56029      56031      56033      56035      56037      56039   
date                                                                           
1980-01-01  17.895370  37.105232  34.901969  29.981843  23.060549  31.983727   
1980-01-02  17.154808  34.340755  36.537107  29.209397  33.287569  30.280188   
1980-01-03  15.489795  32.617469  28.387197  26.546319  36.064677  26.264523   
1980-01-04  13.991697  33.606594  29.985505  30.734602  40.740730  27.300905   
1980-01-05  17.981050  44.482045  27.289303  32.001586  41.432604  30.231203   

state                                       
fips            56041     56043      56045  
date                                        
1980-01-01  37.336613  4.447104  44.677011  
1980-01-02  37.884370  5.147945  36.533748  
1980-01-03  33.982880  4.296923  34.994276  
1980-01-04  43.691094  4.021593  33.944076  
1980-01-05  46.516074  4.872614  44.659092  

[5 rows x 2968 columns]

Do the same for the precipiation dataframe.

In [19]:
# transpose
prcp_df = fips_prcp_df.T
# sort indeces (we know that the first two numbers in the fips code id the state)
prcp_df.sort_index(inplace=True)
# create a state column
prcp_df = prcp_df.assign(state=prcp_df.index.str[:2])
# rename index to fips
prcp_df.index.rename('fips', inplace=True)
# reset index in order to then create a multi-index consisting of state fips and state county fips
prcp_df = prcp_df.reset_index()
prcp_df = prcp_df.set_index(['state', 'fips'])
# re-transpose
prcp_df = prcp_df.T
# make index datetime
prcp_df.index = pd.to_datetime(prcp_df.index)

In [20]:
prcp_df.head()

state                 01                                                       \
fips               01001         01003         01005         01007      01009   
date                                                                            
1980-01-01 -1.206335e-17 -2.616536e-18 -9.750508e-18 -2.854294e-18   0.445504   
1980-01-02 -1.206335e-17 -2.616536e-18 -9.750508e-18 -2.854294e-18   0.074251   
1980-01-03 -1.206335e-17  7.432301e-01 -9.750508e-18  3.799584e+00   0.440340   
1980-01-04  1.946653e+01  8.466931e+00  1.136811e+01  1.177931e+01  13.748179   
1980-01-05 -1.206335e-17  3.362058e-01 -9.750508e-18 -2.854294e-18   0.501705   

state                                                                          \
fips               01011         01013         01015         01017      01019   
date                                                                            
1980-01-01  2.456004e-18 -3.888179e-18  3.069018e+00  5.501198e-18   4.239945   
1980-01-02  2.456004e-18 -3.888179e-18  1.325560e-01  7.885768e-02   0.204112   
1980-01-03  2.456004e-18 -3.888179e-18 -3.074380e-18  5.501198e-18   0.483220   
1980-01-04  1.141392e+01  1.569057e+01  1.765341e+01  7.944504e+00  13.004869   
1980-01-05  2.456004e-18  4.836550e-01  2.120896e-01  1.432760e+00   0.746771   

state           ...             56                                             \
fips            ...          56027     56029    56031     56033         56035   
date            ...                                                             
1980-01-01      ...       0.013794 -0.051540  0.00272 -0.024761  9.062859e-01   
1980-01-02      ...       0.013794  0.224579  0.00272 -0.024761  3.125672e-17   
1980-01-03      ...       0.013794 -0.055619  0.00272  0.016181  3.191147e-01   
1980-01-04      ...       0.013794 -0.067514  0.00272 -0.024761  7.403462e-01   
1980-01-05      ...       0.013794  0.001706  0.00272 -0.024761  1.263694e+00   

state                                                                     
fips               56037     56039         56041     56043         56045  
date                                                                      
1980-01-01 -9.583889e-18  5.038945  1.662734e-01  0.000000 -1.015629e-17  
1980-01-02  7.642688e-01  0.606245  5.957947e-02  0.000000 -1.015629e-17  
1980-01-03 -9.583889e-18  1.648091 -2.734416e-17  0.258283  1.861839e-02  
1980-01-04 -9.583889e-18  4.331765 -2.734416e-17  0.000000 -1.015629e-17  
1980-01-05 -9.583889e-18  7.077837 -2.734416e-17  0.000000 -1.015629e-17  

[5 rows x 2968 columns]

Now we can loop through each of the temperature and precipitation bins to create rows where each row is a day count as described above.

In [21]:
temp_bin_filters = [
    lambda df: (df < 10),
    lambda df: (10 <= df) & (df < 20),
    lambda df: (20 <= df) & (df < 30),
    lambda df: (30 <= df) & (df < 40),
    lambda df: (40 <= df) & (df < 50),
    lambda df: (50 <= df) & (df < 60),
    lambda df: (60 <= df) & (df < 70),
    lambda df: (70 <= df) & (df < 80),
    lambda df: (80 <= df) & (df < 90),
    lambda df: (90 <= df) & (df < 100),
    lambda df: (100 <= df)
]

In [22]:
prcp_bin_filters = [
    lambda df: (df.apply(lambda x: np.isclose(x, 0))),
    lambda df: (0 < df) & (df < 5),
    lambda df: (5 <= df) & (df < 15),
    lambda df: (15 <= df) & (df < 30),
    lambda df: (30 <= df)
]

In [23]:
dfs = list()

In [24]:
for filt in temp_bin_filters:
    # creates boolean dataframe saying whether or not value is in bin
    df = filt(temp_df)
    # groups by year and month and counts number of trues (boolean sum is equivalent to count)
    df = df.groupby([(temp_df.index.year), (temp_df.index.month)]).sum()
    df.index.rename(['year', 'month'], inplace=True)
    dfs.append(df)

In [25]:
for filt in prcp_bin_filters:
    df = filt(prcp_df)
    df = df.groupby([(prcp_df.index.year), (prcp_df.index.month)]).sum()
    df.index.rename(['year', 'month'], inplace=True)
    dfs.append(df)

In [26]:
# let tbi represent the ith temperature bin, pbi represent the ith precipitation bin
bin_col_names = [f'tb{str(i).zfill(2)}' for i in range(1, 12)] + [f'pb{i}' for i in range(1, 6)]
bin_col_names

['tb01',
 'tb02',
 'tb03',
 'tb04',
 'tb05',
 'tb06',
 'tb07',
 'tb08',
 'tb09',
 'tb10',
 'tb11',
 'pb1',
 'pb2',
 'pb3',
 'pb4',
 'pb5']

In [27]:
temp_prcp_df = pd.concat(dfs, axis=1, keys=bin_col_names)
temp_prcp_df.columns = temp_prcp_df.columns.reorder_levels([1, 2, 0])
temp_prcp_df.sort_index(axis=1, inplace=True)
temp_prcp_df.head()

state         01                                                ...     56  \
fips       01001                                                ...  56045   
             pb1   pb2  pb3  pb4  pb5 tb01 tb02  tb03 tb04 tb05 ...   tb02   
year month                                                      ...          
1980 1      17.0   7.0  4.0  2.0  1.0  0.0  0.0   0.0  0.0  2.0 ...    3.0   
     2      21.0   3.0  3.0  2.0  0.0  0.0  3.0   1.0  1.0  4.0 ...    2.0   
     3       8.0   9.0  7.0  4.0  2.0  0.0  1.0   0.0  1.0  2.0 ...    1.0   
     4      16.0   6.0  4.0  2.0  2.0  0.0  0.0   0.0  0.0  0.0 ...    0.0   
     5       0.0  11.0  4.0  0.0  0.0  0.0  0.0  31.0  0.0  0.0 ...    0.0   

state                                                        
fips                                                         
           tb03  tb04  tb05 tb06  tb07  tb08 tb09 tb10 tb11  
year month                                                   
1980 1      4.0   9.0   9.0  2.0   0.0   0.0  0.0  0.0  0.0  
     2      3.0  10.0   9.0  4.0   1.0   0.0  0.0  0.0  0.0  
     3      1.0   4.0  16.0  8.0   0.0   0.0  0.0  0.0  0.0  
     4      0.0   2.0   5.0  5.0  10.0   5.0  3.0  0.0  0.0  
     5      0.0   0.0   2.0  4.0   8.0  13.0  4.0  0.0  0.0  

[5 rows x 47488 columns]

In [28]:
# create lag dataframe
temp_prcp_lag_df = temp_prcp_df.copy()
# simply shift all months up by 1
temp_prcp_lag_df = temp_prcp_lag_df.shift(periods=1, axis=0)
# zero out NaNs generated by shift
temp_prcp_lag_df.loc[temp_prcp_lag_df.index[0]][:] = 0
temp_prcp_lag_df.head()

state         01                                              ...     56       \
fips       01001                                              ...  56045        
             pb1  pb2  pb3  pb4  pb5 tb01 tb02 tb03 tb04 tb05 ...   tb02 tb03   
year month                                                    ...               
1980 1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...    0.0  0.0   
     2      17.0  7.0  4.0  2.0  1.0  0.0  0.0  0.0  0.0  2.0 ...    3.0  4.0   
     3      21.0  3.0  3.0  2.0  0.0  0.0  3.0  1.0  1.0  4.0 ...    2.0  3.0   
     4       8.0  9.0  7.0  4.0  2.0  0.0  1.0  0.0  1.0  2.0 ...    1.0  1.0   
     5      16.0  6.0  4.0  2.0  2.0  0.0  0.0  0.0  0.0  0.0 ...    0.0  0.0   

state                                                  
fips                                                   
            tb04  tb05 tb06  tb07 tb08 tb09 tb10 tb11  
year month                                             
1980 1       0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  
     2       9.0   9.0  2.0   0.0  0.0  0.0  0.0  0.0  
     3      10.0   9.0  4.0   1.0  0.0  0.0  0.0  0.0  
     4       4.0  16.0  8.0   0.0  0.0  0.0  0.0  0.0  
     5       2.0   5.0  5.0  10.0  5.0  3.0  0.0  0.0  

[5 rows x 47488 columns]

In [29]:
# add and "l" before each bin column name to indicate lag column
lag_col_rename_dict = {col_name: f'l{col_name}' for col_name in bin_col_names}

In [30]:
lag_col_rename_dict

{'tb01': 'ltb01',
 'tb02': 'ltb02',
 'tb03': 'ltb03',
 'tb04': 'ltb04',
 'tb05': 'ltb05',
 'tb06': 'ltb06',
 'tb07': 'ltb07',
 'tb08': 'ltb08',
 'tb09': 'ltb09',
 'tb10': 'ltb10',
 'tb11': 'ltb11',
 'pb1': 'lpb1',
 'pb2': 'lpb2',
 'pb3': 'lpb3',
 'pb4': 'lpb4',
 'pb5': 'lpb5'}

In [31]:
temp_prcp_lag_df.rename(columns=lag_col_rename_dict, inplace=True)

In [32]:
temp_prcp_df = pd.concat([temp_prcp_df, temp_prcp_lag_df], axis=1).sort_index(axis=1)

In [33]:
temp_prcp_df.head()

state         01                                                   ...     56  \
fips       01001                                                   ...  56045   
            lpb1 lpb2 lpb3 lpb4 lpb5 ltb01 ltb02 ltb03 ltb04 ltb05 ...   tb02   
year month                                                         ...          
1980 1       0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0 ...    3.0   
     2      17.0  7.0  4.0  2.0  1.0   0.0   0.0   0.0   0.0   2.0 ...    2.0   
     3      21.0  3.0  3.0  2.0  0.0   0.0   3.0   1.0   1.0   4.0 ...    1.0   
     4       8.0  9.0  7.0  4.0  2.0   0.0   1.0   0.0   1.0   2.0 ...    0.0   
     5      16.0  6.0  4.0  2.0  2.0   0.0   0.0   0.0   0.0   0.0 ...    0.0   

state                                                        
fips                                                         
           tb03  tb04  tb05 tb06  tb07  tb08 tb09 tb10 tb11  
year month                                                   
1980 1      4.0   9.0   9.0  2.0   0.0   0.0  0.0  0.0  0.0  
     2      3.0  10.0   9.0  4.0   1.0   0.0  0.0  0.0  0.0  
     3      1.0   4.0  16.0  8.0   0.0   0.0  0.0  0.0  0.0  
     4      0.0   2.0   5.0  5.0  10.0   5.0  3.0  0.0  0.0  
     5      0.0   0.0   2.0  4.0   8.0  13.0  4.0  0.0  0.0  

[5 rows x 94976 columns]

In [34]:
temp_prcp_df.shape

(360, 94976)

In [35]:
# stacking requires a lot of time and memory so stack
# chunks of the dataframe in parallel

In [36]:
def stack(df):
    time.sleep(0)
    return df.stack([0, 1])

In [37]:
year_chunks = list(zip(range(0, 356, 5), range(5, 361, 5)))

In [38]:
num_chunks = len(year_chunks)

In [39]:
dfs = [temp_prcp_df.iloc[start:end, :] for start, end in year_chunks]

In [40]:
rc = ipyparallel.Client()
view = rc.load_balanced_view()

with rc[:].sync_imports():
    import time

importing time on engine(s)


In [41]:
ar = view.map_async(stack, dfs)

In [42]:
# will block the kernel; interrupt the kernel to unblock
ar.wait_interactive()
print(f'Completed {round(ar.serial_time/ar.wall_time, 2)}x faster than serial computation' if ar.ready() else 'Still Running')

  72/72 tasks finished after 1480 s
done
Completed 9.36x faster than serial computation


In [43]:
# ctrl + enter on this cell to see progress without blocking kernel
print(f'Time Elapsed: {datetime.timedelta(seconds=math.ceil(ar.elapsed))}')
print(f'FIPS Codes Completed: {ar.progress}/{num_chunks}')
print(f'Completed {round(ar.serial_time/ar.wall_time, 2)}x faster than serial computation' if ar.ready() else 'Still Running')

Time Elapsed: 0:24:41
FIPS Codes Completed: 72/72
Completed 9.36x faster than serial computation


In [57]:
weather_df = pd.concat(ar)
weather_df.sort_index(inplace=True)

In [60]:
with open('pickles/weather_df.pkl', 'wb') as file:
    pickle.dump(weather_df, file)

In [2]:
with open('pickles/weather_df.pkl', 'rb') as file:
    weather_df = pickle.load(file)

In [3]:
weather_df.head()

lpb1  lpb2  lpb3  lpb4  lpb5  ltb01  ltb02  ltb03  \
year month state fips                                                       
1980 1     01    01001   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01003   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01005   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01007   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01009   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   

                        ltb04  ltb05  ...   tb02  tb03  tb04  tb05  tb06  \
year month state fips                 ...                                  
1980 1     01    01001    0.0    0.0  ...    0.0   0.0   0.0   2.0  16.0   
                 01003    0.0    0.0  ...    0.0   0.0   1.0   1.0   7.0   
                 01005    0.0    0.0  ...    0.0   0.0   0.0   4.0  10.0   
                 01007    0.0    0.0  ...    0.0   0.0   0.0   3.0  17.0   
                 01009    0.0    0.0  ...    0.0   0.0   2.0  15.0  10.0   

                        tb07  tb08  tb09  tb10  tb11  
year month state fips                                 
1980 1     01    01001  12.0   1.0   0.0   0.0   0.0  
                 01003  21.0   1.0   0.0   0.0   0.0  
                 01005  16.0   1.0   0.0   0.0   0.0  
                 01007  10.0   1.0   0.0   0.0   0.0  
                 01009   4.0   0.0   0.0   0.0   0.0  

[5 rows x 32 columns]

In [4]:
weather_df.shape

(1068480, 32)

In [2]:
with open('pickles/all_crime_df.pkl', 'rb') as file:
    all_crime_df = pickle.load(file)

In [3]:
all_crime_df.head()

population  murder  manslaugther  rape  \
year month state fips                                            
1980 1     01    01001     31972.0     0.0           0.0   0.0   
                 01003     78135.0     0.0           0.0   1.0   
                 01005     24132.0     0.0           0.0   1.0   
                 01007      6431.0     0.0           0.0   0.0   
                 01009     36175.0     0.0           0.0   1.0   

                        aggravated assault  simple assault  robbery  burglary  \
year month state fips                                                           
1980 1     01    01001                 7.0             3.0      1.0      29.0   
                 01003                17.0             3.0      5.0      73.0   
                 01005                 5.0            12.0      1.0      14.0   
                 01007                 0.0             0.0      0.0       0.0   
                 01009                 0.0             0.0      1.0      18.0   

                        larceny  vehicle theft  
year month state fips                           
1980 1     01    01001     76.0            0.0  
                 01003     86.0           10.0  
                 01005     44.0            6.0  
                 01007      0.0            0.0  
                 01009      9.0            6.0

In [4]:
# includes population
crime_cols = all_crime_df.columns

In [8]:
weather_df.head()

lpb1  lpb2  lpb3  lpb4  lpb5  ltb01  ltb02  ltb03  \
year month state fips                                                       
1980 1     01    01001   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01003   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01005   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01007   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01009   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   

                        ltb04  ltb05  ...   tb02  tb03  tb04  tb05  tb06  \
year month state fips                 ...                                  
1980 1     01    01001    0.0    0.0  ...    0.0   0.0   0.0   2.0  16.0   
                 01003    0.0    0.0  ...    0.0   0.0   1.0   1.0   7.0   
                 01005    0.0    0.0  ...    0.0   0.0   0.0   4.0  10.0   
                 01007    0.0    0.0  ...    0.0   0.0   0.0   3.0  17.0   
                 01009    0.0    0.0  ...    0.0   0.0   2.0  15.0  10.0   

                        tb07  tb08  tb09  tb10  tb11  
year month state fips                                 
1980 1     01    01001  12.0   1.0   0.0   0.0   0.0  
                 01003  21.0   1.0   0.0   0.0   0.0  
                 01005  16.0   1.0   0.0   0.0   0.0  
                 01007  10.0   1.0   0.0   0.0   0.0  
                 01009   4.0   0.0   0.0   0.0   0.0  

[5 rows x 32 columns]

In [9]:
# TODO: analyze 40,000 record difference between the dataframes
regression_df = weather_df.join(all_crime_df, how='inner')

In [10]:
regression_df.shape

(972000, 42)

In [11]:
# TODO: add custom order to columns
regression_df.head()

lpb1  lpb2  lpb3  lpb4  lpb5  ltb01  ltb02  ltb03  \
year month state fips                                                       
1980 1     01    01001   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01003   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01005   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01007   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01009   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   

                        ltb04  ltb05      ...        population  murder  \
year month state fips                     ...                             
1980 1     01    01001    0.0    0.0      ...           31972.0     0.0   
                 01003    0.0    0.0      ...           78135.0     0.0   
                 01005    0.0    0.0      ...           24132.0     0.0   
                 01007    0.0    0.0      ...            6431.0     0.0   
                 01009    0.0    0.0      ...           36175.0     0.0   

                        manslaugther  rape  aggravated assault  \
year month state fips                                            
1980 1     01    01001           0.0   0.0                 7.0   
                 01003           0.0   1.0                17.0   
                 01005           0.0   1.0                 5.0   
                 01007           0.0   0.0                 0.0   
                 01009           0.0   1.0                 0.0   

                        simple assault  robbery  burglary  larceny  \
year month state fips                                                
1980 1     01    01001             3.0      1.0      29.0     76.0   
                 01003             3.0      5.0      73.0     86.0   
                 01005            12.0      1.0      14.0     44.0   
                 01007             0.0      0.0       0.0      0.0   
                 01009             0.0      1.0      18.0      9.0   

                        vehicle theft  
year month state fips                  
1980 1     01    01001            0.0  
                 01003           10.0  
                 01005            6.0  
                 01007            0.0  
                 01009            6.0  

[5 rows x 42 columns]

In [12]:
# add fixed effects, state-by-month and county-by-year
# all combinations except 1 should be accounted for

In [13]:
single_indexed_regression_df = regression_df.reset_index()

In [14]:
single_indexed_regression_df.head()

,year,month,state,fips,lpb1,lpb2,lpb3,lpb4,lpb5,ltb01,...,population,murder,manslaugther,rape,aggravated assault,simple assault,robbery,burglary,larceny,vehicle theft
0,1980,1,01,01001,0.0,0.0,0.0,0.0,0.0,0.0,...,31972.0,0.0,0.0,0.0,7.0,3.0,1.0,29.0,76.0,0.0
1,1980,1,01,01003,0.0,0.0,0.0,0.0,0.0,0.0,...,78135.0,0.0,0.0,1.0,17.0,3.0,5.0,73.0,86.0,10.0
2,1980,1,01,01005,0.0,0.0,0.0,0.0,0.0,0.0,...,24132.0,0.0,0.0,1.0,5.0,12.0,1.0,14.0,44.0,6.0
3,1980,1,01,01007,0.0,0.0,0.0,0.0,0.0,0.0,...,6431.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1980,1,01,01009,0.0,0.0,0.0,0.0,0.0,0.0,...,36175.0,0.0,0.0,1.0,0.0,0.0,1.0,18.0,9.0,6.0


In [15]:
state_month_series = pd.Series([(state, month) for state, month in zip(single_indexed_regression_df['state'], single_indexed_regression_df['month'])])

In [16]:
fips_year_series = pd.Series([(fips, year) for fips, year in zip(single_indexed_regression_df['fips'], single_indexed_regression_df['year'])])

In [19]:
state_month_dummy_df = pd.get_dummies(state_month_series, drop_first=True)
fips_year_dummy_df = pd.get_dummies(fips_year_series, drop_first=True, sparse=True)
fixed_effects_df = pd.concat([state_month_dummy_df, fips_year_dummy_df], axis=1)

In [20]:
fixed_effects_df.head() 

,"(01, 2)","(01, 3)","(01, 4)","(01, 5)","(01, 6)","(01, 7)","(01, 8)","(01, 9)","(01, 10)","(01, 11)",...,"(56045, 2000)","(56045, 2001)","(56045, 2002)","(56045, 2003)","(56045, 2004)","(56045, 2005)","(56045, 2006)","(56045, 2007)","(56045, 2008)","(56045, 2009)"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
single_indexed_regression_df = pd.concat([single_indexed_regression_df, fixed_effects_df], axis=1)

In [22]:
single_indexed_regression_df.head()

,year,month,state,fips,lpb1,lpb2,lpb3,lpb4,lpb5,ltb01,...,"(56045, 2000)","(56045, 2001)","(56045, 2002)","(56045, 2003)","(56045, 2004)","(56045, 2005)","(56045, 2006)","(56045, 2007)","(56045, 2008)","(56045, 2009)"
0,1980,1,01,01001,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1980,1,01,01003,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1980,1,01,01005,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1980,1,01,01007,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1980,1,01,01009,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
regression_df = single_indexed_regression_df.set_index(['year', 'month', 'state', 'fips'])

In [25]:
regression_df.head()

lpb1  lpb2  lpb3  lpb4  lpb5  ltb01  ltb02  ltb03  \
year month state fips                                                       
1980 1     01    01001   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01003   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01005   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01007   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01009   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   

                        ltb04  ltb05      ...        (56045, 2000)  \
year month state fips                     ...                        
1980 1     01    01001    0.0    0.0      ...                    0   
                 01003    0.0    0.0      ...                    0   
                 01005    0.0    0.0      ...                    0   
                 01007    0.0    0.0      ...                    0   
                 01009    0.0    0.0      ...                    0   

                        (56045, 2001)  (56045, 2002)  (56045, 2003)  \
year month state fips                                                 
1980 1     01    01001              0              0              0   
                 01003              0              0              0   
                 01005              0              0              0   
                 01007              0              0              0   
                 01009              0              0              0   

                        (56045, 2004)  (56045, 2005)  (56045, 2006)  \
year month state fips                                                 
1980 1     01    01001              0              0              0   
                 01003              0              0              0   
                 01005              0              0              0   
                 01007              0              0              0   
                 01009              0              0              0   

                        (56045, 2007)  (56045, 2008)  (56045, 2009)  
year month state fips                                                
1980 1     01    01001              0              0              0  
                 01003              0              0              0  
                 01005              0              0              0  
                 01007              0              0              0  
                 01009              0              0              0  

[5 rows x 81628 columns]

In [26]:
with open('pickles/regression_df.pkl', 'wb') as file:
    pickle.dump(regression_df, file)

In [5]:
with open('pickles/regression_df.pkl', 'rb') as file:
    regression_df = pickle.load(file)

In [6]:
regression_without_responses_df = regression_df.drop(columns=crime_cols)

In [7]:
regression_without_responses_df.head()

lpb1  lpb2  lpb3  lpb4  lpb5  ltb01  ltb02  ltb03  \
year month state fips                                                       
1980 1     01    01001   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01003   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01005   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01007   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
                 01009   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0   

                        ltb04  ltb05      ...        (56045, 2000)  \
year month state fips                     ...                        
1980 1     01    01001    0.0    0.0      ...                    0   
                 01003    0.0    0.0      ...                    0   
                 01005    0.0    0.0      ...                    0   
                 01007    0.0    0.0      ...                    0   
                 01009    0.0    0.0      ...                    0   

                        (56045, 2001)  (56045, 2002)  (56045, 2003)  \
year month state fips                                                 
1980 1     01    01001              0              0              0   
                 01003              0              0              0   
                 01005              0              0              0   
                 01007              0              0              0   
                 01009              0              0              0   

                        (56045, 2004)  (56045, 2005)  (56045, 2006)  \
year month state fips                                                 
1980 1     01    01001              0              0              0   
                 01003              0              0              0   
                 01005              0              0              0   
                 01007              0              0              0   
                 01009              0              0              0   

                        (56045, 2007)  (56045, 2008)  (56045, 2009)  
year month state fips                                                
1980 1     01    01001              0              0              0  
                 01003              0              0              0  
                 01005              0              0              0  
                 01007              0              0              0  
                 01009              0              0              0  

[5 rows x 81618 columns]

In [8]:
# model
mod = Poisson(regression_df['murder'][:5], regression_without_responses_df.head().to_sparse())

In [9]:
# result
res = mod.fit()

C:\Users\avi\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1019: RuntimeWarning: divide by zero encountered in log
  params = [np.log(const)]


MemoryError: 

In [ ]:
res.params